In [7]:
# Correct and Minimal S-AES Implementation

# S-Box and its inverse
SBOX = {'0000': '1001', '0001': '0100', '0010': '1010', '0011': '1011', '0100': '1101', 
        '0101': '0001', '0110': '1000', '0111': '0101', '1000': '0110', '1001': '0010', 
        '1010': '0000', '1011': '0011', '1100': '1100', '1101': '1110', '1110': '1111', '1111': '0111'}
INV_SBOX = {v: k for k, v in SBOX.items()}

# Core functions
def xor(a, b): return format(int(a, 2) ^ int(b, 2), f'0{len(a)}b')
def sub_nibbles(state): return ''.join(SBOX[state[i:i+4]] for i in range(0, len(state), 4))
def inv_sub_nibbles(state): return ''.join(INV_SBOX[state[i:i+4]] for i in range(0, len(state), 4))
def shift_rows(state): return state[:4] + state[12:16] + state[8:12] + state[4:8]

def gen_keys(key):
    def g(w, rc):
        rotated = w[4:] + w[:4]
        subbed = sub_nibbles(rotated)
        return xor(subbed, rc)
    
    w0, w1 = key[:8], key[8:]
    w2 = xor(w0, g(w1, '10000000'))
    w3 = xor(w2, w1)
    w4 = xor(w2, g(w3, '00110000'))
    w5 = xor(w4, w3)
    
    return [w0 + w1, w2 + w3, w4 + w5]

def mul_gf2(a, b):
    """Multiply two polynomials in GF(2^4) with irreducible polynomial x^4 + x + 1"""
    p = 0
    a_val = int(a, 2)
    b_val = int(b, 2)
    
    for i in range(4):
        if (b_val & 1) == 1:
            p ^= a_val
        
        high_bit = a_val & 8
        a_val = (a_val << 1) & 15
        if high_bit:
            a_val ^= 3  # x^4 = x + 1
        
        b_val >>= 1
        
    return format(p, '04b')

def mix_columns(state):
    """Mix columns operation with matrix [[1, 4], [4, 1]]"""
    s0 = state[:4]
    s1 = state[4:8]
    s2 = state[8:12]
    s3 = state[12:16]
    
    # Convert 4 (0100) to polynomial notation and use GF multiplication
    s0_new = xor(s0, mul_gf2('0100', s2))
    s1_new = xor(s1, mul_gf2('0100', s3))
    s2_new = xor(mul_gf2('0100', s0), s2)
    s3_new = xor(mul_gf2('0100', s1), s3)
    
    return s0_new + s1_new + s2_new + s3_new

def inv_mix_columns(state):
    """Inverse mix columns operation with matrix [[9, 2], [2, 9]]"""
    s0 = state[:4]
    s1 = state[4:8]
    s2 = state[8:12]
    s3 = state[12:16]
    
    # 9 = 1001 and 2 = 0010 in binary
    s0_new = xor(mul_gf2('1001', s0), mul_gf2('0010', s2))
    s1_new = xor(mul_gf2('1001', s1), mul_gf2('0010', s3))
    s2_new = xor(mul_gf2('0010', s0), mul_gf2('1001', s2))
    s3_new = xor(mul_gf2('0010', s1), mul_gf2('1001', s3))
    
    return s0_new + s1_new + s2_new + s3_new

# Main encryption/decryption functions
def encrypt(plaintext, key):
    keys = gen_keys(key)
    
    state = xor(plaintext, keys[0])
    
    # Round 1
    state = sub_nibbles(state)
    state = shift_rows(state)
    state = mix_columns(state)
    state = xor(state, keys[1])
    
    # Round 2
    state = sub_nibbles(state)
    state = shift_rows(state)
    state = xor(state, keys[2])
    
    return state

def decrypt(ciphertext, key):
    keys = gen_keys(key)
    
    state = xor(ciphertext, keys[2])
    
    # Round 1 - inverse
    state = shift_rows(state)
    state = inv_sub_nibbles(state)
    state = xor(state, keys[1])
    state = inv_mix_columns(state)
    
    # Round 2 - inverse
    state = shift_rows(state)
    state = inv_sub_nibbles(state)
    state = xor(state, keys[0])
    
    return state

# Test the implementation
if __name__ == "__main__":
    plaintext = '0110111101101011'
    key = '1010001110100011'
    
    ciphertext = encrypt(plaintext, key)
    decrypted = decrypt(ciphertext, key)
    
    print(f"Plaintext : {plaintext}")
    print(f"Key       : {key}")
    print(f"Ciphertext: {ciphertext}")
    print(f"Decrypted : {decrypted}")
    print(f"Match     : {'Yes' if plaintext == decrypted else 'No'}")

Plaintext : 0110111101101011
Key       : 1010001110100011
Ciphertext: 1010110100000001
Decrypted : 0110111101101011
Match     : Yes
